# SQL Querry

Initital Test Query to Examin the Dataset

In [ ]:
CREATE TABLE `CB_2107.SNOMED` AS 
SELECT * FROM `CB_LOOKUPS.tbl_EFI2_Codelist`
WHERE (Codedescription LIKE "%bronchitis")
#Querry 1 - was run on simple dissorder to see code dupliacation exist

In [ ]:
CREATE TABLE `CB_2107.SNOMED_FLU` AS 
SELECT * FROM `CB_LOOKUPS.tbl_EFI2_Codelist`
WHERE (SNOMEDCT_CONCEPTID LIKE "%6142004 ")
##### This code was run to find all the types of flue codes that are avaialble on the SNOMED data list for FLU

In [ ]:
CREATE TABLE `CB_2107.People_Details` AS
SELECT * FROM `CB_FDM_PrimaryCare_V8.person`

WHERE (year_of_birth) > 1950
  AND death_datetime IS NULL;
### This code is run to create a table from the entire dataset, that shows only people born after the year 1950, 
#the dataset contains data that is as Old as people born in the 1900, however for the purpose if this reserch
#I have decided to unly include the poeple from 1950

In [ ]:
CREATE TABLE `CB_2107.death_Count_null_per_Year` AS
SELECT (year_of_birth) AS birth_year,
  COUNT(*) AS null_death_count
FROM
   `CB_FDM_PrimaryCare_V8.person`
WHERE
  death_datetime IS NULL
GROUP BY (year_of_birth)
ORDER BY (year_of_birth);
### This Query was run to see how many death data was captured and how many null 
#values per data per year, this was done to check the missinging value feature 

In [ ]:
SELECT (year_of_birth) AS birth_year,
  COUNT(*) AS null_birth_count
FROM
   `CB_FDM_PrimaryCare_V8.person`
WHERE
  year_of_birth IS NULL
GROUP BY (year_of_birth)
ORDER BY (year_of_birth);
#### There are no null values for Date of birth - That data is captured perfectly 

In [ ]:
SELECT
  COUNT(*) AS ethnicity_source_value
FROM
  `CB_FDM_PrimaryCare_V8.person`
WHERE
  ethnicity_source_value = 'Unknown/Refuse to say';
#This category should have been split into two - to analyse if people did not know their ethnicity or 
# if they refused to say? There is no way to distinguish these details. Also, was this data recorded on the day the patient was 
# registered? or was it captured later and only entered as "Unknow/refuse to say" because the people who recorded the
#did not know then? There should be two separate sections, and this field must be made mandatory to 
#understand the origins of people, especially considering how diverse the country is and how much this data could mean. 
#Also, when this data gets carried forward, there is a possibility that there are two separate categories. 
#but since this data is not separated.. which category will this fit into? If it is placed in the wrong category, will that miss 
#lead research? 

In [ ]:
SELECT 
p.person_id,
a.care_site_id,
m.nameofmedication,
c.care_site_name
FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.person` AS p
LEFT JOIN `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srappointment` AS a ON p.person_id = a.person_id
LEFT JOIN `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srprimarycaremedication` AS m ON p.person_id = m.person_id
LEFT JOIN `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.care_site` AS c ON a.care_site_id = c.care_site_id
LIMIT 1000;


Find out the count CTV3 codes that are assigned according to thier care site id
number of record per CTV3 code per Care site ID -
Then find the code for Broncitis through open code list
see which GP practice used which of the codes the most often..
see if there is a problem? - If a certain code is used more often then others?

In [ ]:
CREATE TABLE `CB_2107.MassTest` AS
  SELECT
  P.person_id,
  C.person_id AS srcode_person_id,
  C.care_site_id,
  Ca.care_site_name,
  C.ctv3code,
  C.ctv3text,
  C.dateevent

FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.person` AS P
JOIN
  `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srcode` AS C ON P.person_id = C.person_id
JOIN
  `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.care_site`  AS Ca ON C.care_site_id = Ca.care_site_id;
# Testing to join the tables based on Primary Key 

In [ ]:
CREATE TABLE `CB_2107.Aggregate_Data_MassTest` AS
SELECT
  C.care_site_id,
  C.ctv3code,
  C.ctv3text,
  COUNT(*) AS RecordCount
FROM
  `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.person` AS P
JOIN
   `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srcode` AS C ON P.person_id = C.person_id
JOIN
  `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.care_site`  AS Ca ON C.care_site_id = Ca.care_site_id
GROUP BY
  C.care_site_id,
  C.ctv3code,
  C.ctv3text
ORDER BY
  C.care_site_id,
  C.ctv3code;

#### group data in the above table by  record count  CareSiteID and CTV3 Code - This shows the care site ID, the CTV Code 
# and the number of patients per code per care site - This shows if a certain practice uses a cerain code more regularly 
# than the other practices THIS IS FOR ALL ISSUES

In [ ]:
CREATE TABLE `CB_2107.Bronchitis` AS
SELECT ctv3code,ctv3text,person_id,care_site_id
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE ctv3text LIKE '%bronchitis%';
# Query to get all the patients that have bronchitis 

In [ ]:
CREATE TABLE `CB_2107.Bronchitis_Test` AS
SELECT ctv3code,
ctv3text,
person_id,
care_site_id,
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE ctv3text LIKE '%bronchitis%'
GROUP BY
  care_site_id,
  ctv3code,
  ctv3text,
  person_id;
#Table created - May not be a useful code - coz nothing is aggregated - it just pulls each indevidual person that has
#bronchitis per caresite id 

In [ ]:
CREATE TABLE `CB_2107.Bronchitis_Aggregated_by_CareSite` AS
SELECT
  care_site_id,
  ctv3code,
  ctv3text,
  COUNT(DISTINCT person_id) AS PersonCount
FROM
  `CB_FDM_PrimaryCare_V8.tbl_srcode`

WHERE
  ctv3text LIKE '%bronchitis%'

GROUP BY
  care_site_id,
  ctv3code,
  ctv3text
ORDER BY
  care_site_id,
  ctv3text;

# Get the numebr of records at each site, per type of bronchitis 
#A table has ctv3code, ctv3text, person_id, care_site_id - i want to know the number of people 
#diagnosed with each disease in each care_site_id , where the code meaning contains the term "bronchitis"

In [ ]:
CREATE TABLE `CB_2107.Dates_notMatch_All` AS
SELECT care_site_id,
  CASE WHEN dateeventrecorded = dateevent THEN 'Match'
       ELSE 'DontMatch' END AS RecordCategory,
  COUNT(*) AS RecordCount
FROM
  `CB_FDM_PrimaryCare_V8.tbl_srcode`
GROUP BY
  dateevent,
  care_site_id,
  dateeventrecorded;

    ## A table has Table had incidenthappened and incidentrecorded, for each record these two dates may not be exactly the same,
    #Write SQL to get the number of records that have matching and mismatching records

In [ ]:
CREATE TABLE `CB_2107.Dates_notMatch_All_groupedbyCaresite` AS
SELECT
  care_site_id,
  SUM(CASE WHEN dateevent = dateeventrecorded THEN 1 ELSE 0 END) AS matching_records,
  SUM(CASE WHEN dateevent <> dateeventrecorded THEN 1 ELSE 0 END) AS mismatching_records
FROM
  `CB_FDM_PrimaryCare_V8.tbl_srcode`
GROUP BY
  care_site_id;
    # Total number of Matched and not matched number of Date event happend and date recorded per care_site_id
    # This needs to be matched with Care_site table to get the care site Name 

In [ ]:
CREATE TABLE `CB_2107.Bronchitis_Care_Site_Name` AS
SELECT x.*, y.care_site_name
FROM `CB_FDM_PrimaryCare_V8.care_site` y
INNER JOIN `CB_2107.Bronchitis_Aggregated_by_CareSite`x ON
 x.care_site_id = y.care_site_id
#Inner join to join get the care_site Name and id Matched 

In [ ]:
CREATE TABLE `CB_2107.Bronchitis_Care_Site_Acute_Bron` AS
SELECT *
FROM `CB_2107.Bronchitis_Care_Site_Name`
WHERE ctv3text LIKE '%Acute bronchitis%';
# Only care sites with Accute Bronchitis were selected to see if codes are selcted more often then others 

# Null SNOMED VALUES 

In [ ]:
CREATE TABLE `CB_2107.NULL_SNOMED_CODE_PER_YEAR_SITE` AS
SELECT
    care_site_id,
    EXTRACT(YEAR FROM dateeventrecorded) AS Year,
    COUNT(*) AS NULL_SNOMED_CODE
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE care_site_id IS NOT NULL AND snomedcode IS NULL
GROUP BY care_site_id, Year
ORDER BY care_site_id, Year;

In [ ]:
CREATE TABLE `CB_2107.NULL_SNOMED_CODE_PER_YEAR_SITE_JOINED` AS

SELECT A.*, B.care_site_name
FROM `yhcr-prd-phm-bia-core.CB_2107.NULL_SNOMED_CODE_PER_YEAR_SITE` A
INNER JOIN `CB_FDM_PrimaryCare_V8.care_site`B ON A.care_site_id = B.care_site_id;


# Bronchitis Test Codes

In [ ]:
CREATE TABLE `CB_2107.ALL_Bronchitis` AS
SELECT person_id,dateeventrecorded,dateevent,ctv3code,ctv3text,care_site_id
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (dateeventrecorded > CAST('2020-01-01' AS DATE)) 
AND (ctv3text LIKE '% bronchitis%' OR ctv3text LIKE '% Bronchitis%');

In [ ]:
# Get all data for bronc with all details and create a new table where the date is greater than 2020 Jan 
#and only conatins Acute Bron
CREATE TABLE `CB_2107.Acute Bronchitis` AS
SELECT *
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (dateeventrecorded > CAST('2020-01-01' AS DATE)) 
AND (ctv3text LIKE '%Acute bronchitis%');


In [ ]:
CREATE TABLE `CB_2107.Acute Bronchitis_Simplified` AS
SELECT person_id,dateeventrecorded,dateevent,ctv3code,ctv3text,care_site_id
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (dateeventrecorded > CAST('2020-01-01' AS DATE)) 
AND (ctv3text LIKE '%Acute bronchitis%');
# Simplified to only include certain fields 

In [ ]:
CREATE TABLE `CB_2107.Acute Bronchitis_Simplified_SNOMED` AS
SELECT a.*, b.SNOMEDCode
FROM `CB_2107.Acute Bronchitis_Simplified`a 
INNER JOIN  `CB_LOOKUPS.tbl_CTV3ToSnomed_Map`b ON a.CTV3Code = b.CTV3Code;
# SNOMED Codes are mapped 


In [ ]:
CREATE TABLE `CB_2107.Acute Bronchitis_Simplified_Match` AS
SELECT
    SUM(CASE WHEN dateevent = dateeventrecorded THEN 1 ELSE 0 END) AS DateMatched,
    SUM(CASE WHEN dateevent <> dateeventrecorded THEN 1 ELSE 0 END) AS DateNotMatched
FROM `CB_2107.Acute Bronchitis_Simplified_SNOMED`;
# Query to check the match and do not Match field 

In [ ]:
CREATE TABLE `CB_2107.Acute Bronchitis_Simplified_2021` AS
SELECT person_id,dateeventrecorded,dateevent,ctv3code,ctv3text,care_site_id
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (dateeventrecorded > CAST('2021-01-01' AS DATE)) 
AND (ctv3text LIKE '%Acute bronchitis%');

In [ ]:
CREATE TABLE `CB_2107.Acute Bronchitis_Simplified_Match_2021` AS
SELECT
    SUM(CASE WHEN dateevent = dateeventrecorded THEN 1 ELSE 0 END) AS DateMatched,
    SUM(CASE WHEN dateevent <> dateeventrecorded THEN 1 ELSE 0 END) AS DateNotMatched
FROM `CB_2107.Acute Bronchitis_Simplified_2021`;
# Query to check the match and do not Match field 

In [ ]:
SELECT ctv3code,ctv3text, COUNT(*) AS NumberOfRecords
FROM `CB_2107.ALL_Bronchitis`
GROUP BY ctv3code,ctv3text;
# Query to check all the CTV3 codes for Bronchitis and thier number of records 

In [ ]:
CREATE TABLE `CB_2107.All Not Match` AS
SELECT
    SUM(CASE WHEN dateevent = dateeventrecorded THEN 1 ELSE 0 END) AS DateMatched,
    SUM(CASE WHEN dateevent <> dateeventrecorded THEN 1 ELSE 0 END) AS DateNotMatched
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`;
# Match / Not Match for All Data 

In [ ]:
CREATE TABLE `CB_2107.Acute Bronc agg by site` AS
SELECT
  care_site_id,
  ctv3code,
  ctv3text,
  COUNT(DISTINCT person_id) AS PersonCount
FROM
  `CB_2107.Acute Bronchitis_Simplified_SNOMED`

GROUP BY
  care_site_id,
  ctv3code,
  ctv3text
ORDER BY
  care_site_id,
  ctv3text;
# Acute Bronc aggregated by Site

In [ ]:
CREATE TABLE `CB_2107.SNOMED_CTV3_Mapping issues` AS
SELECT SNOMEDCode,COUNT(CTV3Code) AS CTV3CodeCount
FROM `CB_LOOKUPS.tbl_CTV3ToSnomed_Map`
GROUP BY SNOMEDCode
HAVING COUNT(DISTINCT CTV3Code) > 1;
#Table A has Ctcode and medCode - Multiple Ctcode can be mapped into a single medCode , 
#SQL query to  Print all the Medcode where a more than 1 ctv code has been assigned to, 
#alsong with a recod count of the number of ctv code assigned to each code 

In [ ]:
SELECT
    SNOMEDCode,
    COUNT(CTV3Code) AS CTV3Code_count
FROM
    `CB_LOOKUPS.tbl_CTV3ToSnomed_Map`
GROUP BY
    SNOMEDCode
HAVING
    COUNT(CTV3Code) > 1
ORDER BY
    CTV3Code_count DESC, SNOMEDCode DESC
LIMIT 20;
# Same things as above, but only the first 20 recods with the highest count 

In [ ]:
CREATE TABLE `CB_2107.SNOMED_CTV3_Mapping issues_20_Codes` AS
SELECT
    SNOMEDCode,CTV3Code,
    
FROM
    `CB_LOOKUPS.tbl_CTV3ToSnomed_Map`
WHERE
    SNOMEDCode IN ('52684005', '214640008', '34552002','3160009','3160009','418019003', '127349007','127350007','129675007','399269003','217176002','716481000000102','473023007','166001','88817006','90688005','416098002','404684003','71388002','242089005','91388009') 
GROUP BY
    SNOMEDCode,CTV3Code;
# SNOMED and CTV3 codes for the top 20 codes with highest CTV3 Mapping 
# there is an issue with dataset, this table ot the other tables in dataset that provides similar details which one is correct?  

In [ ]:
CREATE TABLE `CB_2107.SNOMED_CTV3_Mapping issues_concept code-ID` AS
SELECT SNOMEDCT_CONCEPTID,COUNT(CTV3) AS CTV3CodeCount
FROM `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist`
GROUP BY SNOMEDCT_CONCEPTID
HAVING COUNT(DISTINCT CTV3) > 1;
# Using the EF1_2 Codelist to draw codes that are matched to multiple snomed/CTV3

In [ ]:
SELECT
    SNOMEDCT_CONCEPTID,
    STRING_AGG(CTV3, ', ') AS CTV3,
    STRING_AGG(Codedescription, ', ') AS Codedescription
FROM
    `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist`
GROUP BY
    SNOMEDCT_CONCEPTID;
 #NOT THE DESIRED RESULT 

In [ ]:
SELECT
    CTV3Code,CTV3Desc
FROM
    `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_CTV3Codes_Lookup` 
WHERE
    CTV3Code LIKE '%Eu102%'OR CTV3Code LIKE '%Eu107%'OR CTV3Code LIKE '%Eu10z%'
    
#Shows the CTV3 code and thier decription
#This code and the code below shows that multiple CTV3 codes such as shows int he document have been mapped into single CTV3.
# Anothe issue some use SNOMED some use CTV3 - the ones that used CTV3, is all code is not accounted for
# some info will be missed. 


In [ ]:
SELECT
    CTV3Code,SNOMEDCode
FROM
    `CB_2107.SNOMED_CTV3_Mapping issues_20_Codes`
WHERE
    CTV3Code LIKE '%Eu10%';
    #Shows the SNOMED CODE AND CTV3 Code- Check Desc Above 

In [ ]:
SELECT
    CTV3,Codedescription,SNOMEDCT_CONCEPTID
FROM
    `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist`
WHERE
    SNOMEDCT_CONCEPTID = '191448002';
# This code tests for all the codes that have multiple CTV3 codes mapped into them. this table is found SNOMED_CTV3_Mapping
#iSSUES cONCEPT CODE_id

# Covid SQL

In [ ]:
CREATE TABLE `CB_2107.COVID_CODE_INTRO` AS
SELECT * FROM `CB_CDM_VOCAB.concept`
WHERE (valid_end_date > CAST('1950-01-01' AS DATE)) AND 
(concept_name LIKE '%Coronavirus%' OR concept_name LIKE '%Coronavirus%')
ORDER BY valid_end_date ASC

# Shows all the people that have Coronavirus in thier GP records 
# Multiple different codeing systems are used at each GP
# THIS CODE IN SNOMED WAS INTODUCED IN 2020 MAY
# what was this before- Covid stated in early 2020 ? Many records have been missed?

In [ ]:
CREATE TABLE `CB_2107.COVID_CODE_DIFFERENT` AS
SELECT vocabulary_id, COUNT(*) AS count
FROM `CB_2107.COVID_CODE_INTRO`
GROUP BY vocabulary_id;
# This query has all the Coding systems, and how many codes under each system since 1950, with SNOMED having the second
#highest - this includes the Obervation and medication and all categories - Just to get an Idea (Initial analysis)

In [ ]:
CREATE TABLE `CB_2107.COVID_SNOMED` AS
SELECT *
FROM `CB_CDM_VOCAB.concept`
WHERE vocabulary_id = 'SNOMED'
  AND concept_name LIKE '%Coronavirus %'
  AND valid_end_date > CAST('1950-01-01' AS DATE);

# This query only gets the Concept code for COVID that have SNOMED id 

In [ ]:
CREATE TABLE `CB_2107.COVID_SNOMED_INVALID` AS
SELECT *
FROM `CB_2107.COVID_SNOMED`
WHERE invalid_reason != 'null';
#Creates A LIST of invalid codes for Covid for SNOMED

In [ ]:
CREATE TABLE `CB_2107.COVID_SNOMED_VALID` AS
SELECT *
FROM `CB_2107.COVID_SNOMED`
WHERE invalid_reason IS NULL ;
#Creates A LIST of valid codes for Covid for SNOMED

In [ ]:
CREATE TABLE `CB_2107.COVID_Cases` AS
SELECT
    *
    
FROM
    `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE
    SNOMEDCode IN ('120814005', '186747009', '204351000000100','906711000000107',
                   '1029481000000103','1008541000000105', '933791000000101','817221000000104',
                   '817111000000102','225631000001109','225591000001109','225581000001107','225681000001108',
                   '226631000001102','225571000001105');
#List of people with Covid - Only checking for SNOMED Codes

In [ ]:
CREATE TABLE `CB_2107.COVID_SNOMED_JOINED` AS
SELECT *
FROM `CB_2107.COVID_SNOMED_VALID`
INNER JOIN `CB_2107.COVID_Cases`  ON CAST(`CB_2107.COVID_SNOMED_VALID`.concept_code AS STRING) = `CB_2107.COVID_Cases` .snomedcode;
# The SNOMED codes have noe been Matched for thier description 

In [ ]:
CREATE TABLE `CB_2107.COVID_CODE_START_Date` AS
SELECT
    snomedcode,concept_name,valid_start_date,
    EXTRACT(YEAR FROM dateevent) AS Year,
    EXTRACT(MONTH FROM dateevent) AS Month,
    COUNT(*) AS NumberOfRecords
FROM `CB_2107.COVID_SNOMED_JOINED`
GROUP BY snomedcode, Year, Month, concept_name,valid_start_date
ORDER BY snomedcode, Year, Month,concept_name,valid_start_date;
# Start Date of all Covid Codes, and when they were used the most 
# Although the Codes 

In [ ]:
CREATE TABLE `CB_2107.COVID_BY_CARESITE_BY_MMONTH_YEAR` AS

SELECT
    care_site_id,
    EXTRACT(YEAR FROM dateevent) AS Year,
    EXTRACT(MONTH FROM dateevent) AS Month,
    COUNT(*) AS NumberOfRecords
FROM `CB_2107.SNOMED_NAME_MAPPED`
GROUP BY care_site_id, Year, Month
ORDER BY care_site_id, Year, Month;


#Covid case aggregated by Year and Month per Caresite ID 

In [ ]:
CREATE TABLE `CB_2107.COVID_BY_CARESITE_name_BY_MONTH_YEAR` AS
SELECT A.*, B.care_site_name,location_id,place_of_service_concept_id
FROM `CB_2107.COVID_BY_CARESITE_BY_MMONTH_YEAR` A
INNER JOIN `CB_FDM_PrimaryCare_V8.care_site` B
ON A.care_site_id = B.care_site_id;
#CareSite Name joined from Primarycare_CareSite 

In [ ]:
CREATE TABLE `CB_2107.COVID_BY_CARESITE_name_Aggrigated` AS
SELECT
    care_site_name,
    concept_name,
    EXTRACT(DATE FROM dateevent) AS Year,
    COUNT(*) AS NumberOfCases
FROM `yhcr-prd-phm-bia-core.CB_2107.SNOMED_NAME_MAPPED_CareSiteName`
GROUP BY care_site_name, concept_name, Year
ORDER BY care_site_name, Year, concept_name;


# COVID Codes Part II

In [ ]:
CREATE TABLE `CB_2107.NEW_ALL_COVID_CASES` AS

SELECT *
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (ctv3text LIKE '%COVID%' OR ctv3text LIKE '%Coronavirus%'OR ctv3text LIKE '%Long COVID%')
#table of all cases of Covid from the Peopl Table 

In [ ]:
CREATE TABLE `CB_2107.NEW_ALL_COVID_CASES_2017` AS
SELECT * FROM `CB_2107.NEW_ALL_COVID_CASES`
WHERE (dateevent > CAST('2017-12-30' AS DATE)) 
#cASES FROM 2017

In [ ]:
CREATE TABLE `CB_2107.NEW_ALL_COVID_CASES_Grouped_CTV3` AS
SELECT ctv3code, ctv3text,snomedcode, COUNT(*) AS CountOfRecords
FROM `CB_2107.NEW_ALL_COVID_CASES`
GROUP BY ctv3code, ctv3text,snomedcode;
# Group the table by CTV3 codes 

In [ ]:
CREATE TABLE `CB_2107.NEW_COVID_SNOMED` AS
SELECT *
FROM `CB_CDM_VOCAB.concept`
WHERE vocabulary_id = 'SNOMED'
  AND (concept_name LIKE '%Coronavirus %' OR concept_name LIKE '%COVID %'OR concept_name LIKE '%coronavirus %'OR concept_name LIKE '%corona %' )
  AND Invalid_reason  IS NULL ;
# A list of all Covid Codes from SNOMED Concept 

In [ ]:
CREATE TABLE `CB_2107.NEW_COVID_SNOMED_GROUPED` AS

SELECT valid_start_date, COUNT(*) AS NumberOfRecords
FROM `CB_2107.NEW_COVID_SNOMED`
GROUP BY valid_start_date;
# Number of codes grouped by thier release year 

In [ ]:
SELECT *
FROM `CB_2107.NEW_ALL_COVID_CASES_2017`
WHERE ctv3code = 'X731E';


# Influenza SQL

In [ ]:
#CREATE TABLE `CB_2107.Influenza_Cases` AS
SELECT
    *
    
FROM
    `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE
    SNOMEDCode IN ('10629191000119100', '719865001', '719590007','906711000000107',
                   '408687004','408685007', '407496005','441119003',
                   '195726000','27475006','230188005','59221008','30270006',
                   '64880000','64917006','68949000','78046005','78911000','85832003',
                   '70233007', '35377009', '192721000','10809006','932221000000103',
                   '1033091000000109','1033111000000104','1033051000000101','1033071000000105');
    # Creating a list of people who has Influenza using the valid codes 
    #Table deleted 

In [ ]:
CREATE TABLE `CB_2107.Influenza_Cases_2019` AS

SELECT
    *
    
FROM
    `CB_FDM_PrimaryCare_V8.tbl_srcode`

WHERE  (dateevent > CAST('2019-01-01' AS DATE)) AND 
ctv3text  LIKE '%Influenza%' AND ctv3text  LIKE '%flu%' AND
  (ctv3text NOT LIKE '%vacc%' AND ctv3text NOT LIKE '%SMS%' AND ctv3text NOT LIKE '%immunisation%'
  AND ctv3text NOT LIKE '%invite %'AND ctv3text NOT LIKE '%Vaccine%'); 
    
    # All the cases with inlfuenza from 2019 and Above 

In [ ]:
CREATE TABLE `CB_2107.Influenza_Cases_2019_Record_Count` AS
SELECT snomedcode, COUNT(*) AS RecordCount
FROM `CB_2107.Influenza_Cases_2019`
GROUP BY snomedcode
ORDER BY snomedcode;
# aLL THE RECORD FOR Inlfuelza code FROM pERSON TO snomed

In [ ]:
CREATE TABLE `CB_2107.Influenza_Cases_2019_Record_Count_joined` AS
SELECT A.*,B.*
FROM `CB_2107.Influenza_Cases_2019_Record_Count` A
INNER JOIN `CB_CDM_VOCAB.concept`B
ON A.snomedcode = B.concept_code;
# Joined table for all the codes extracted using the srcode first - where condidtion and everything is mixed - 
#From person to SNOMED 

In [ ]:
SELECT
    *
    
FROM
    `CB_CDM_V1_50k_Random.concept`
WHERE
    concept_code IN ('1033091000000109', '195878008', '195929004','315642008',
                   '359351000000100','407479009', '407480007','441049004',
                   '441345003','442696006','444426005','511781000000101','55014007',
                   '6142004','711330007','95891005');
    
    # All the matching codes from the cases table to match the Concept table from the rocord count table 
    # Checking from pERSON TABLE TO snomed 
    

# Influenza Code Part II - Testing inclusion criteria

In [ ]:
CREATE TABLE `CB_2107.Influenza_Active_Codes` AS

     SELECT *
FROM `CB_CDM_VOCAB.concept`

  WHERE concept_name LIKE '%Influenza%'
 AND domain_id = 'Condition' AND vocabulary_id = 'SNOMED'
  AND invalid_reason IS  NULL; 
# Checking for valid SNOMED cases with Influenza - Has many codes 

In [ ]:
#Checing from SNOMED to person
CREATE TABLE `CB_2107.Influenza_Cases_snomed_Cases` AS
SELECT
    *
    
FROM
    `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE
    SNOMEDCode IN ('10628911000119103', '10674911000119108', '142951000119106','10629351000119108',
                   '711334003','10685111000119102', '328531000119104','10677711000119101',
                   '772839003','772810003','772828001','866126000','719865001',
                   '719590007','453931000124108','434931000124106','435051000124104','707448003','961000124101',
                   '70233007', '35377009', '192721000','10809006','932221000000103',
                   '941000124100','951000124103','921000124107','931000124105',
                   
                   '442438000','442696006','1149091008',
                   '738276008','471361000124100','713083002','450716003','450715004','74644004',
                   '195923003', '43692000', '41269000','195929004','24662006',
                   '315642008','95891005','195924009','81524006',
                   
                   '6142004','61700007','1033091000000109',
                   '1033051000000101','1033071000000105','1033111000000104') AND (dateevent > CAST('2019-01-01' AS DATE));
    # tHIS METHOD WAS ALSO TRIED - BUT DID NOT GIVE A LOT OF VARIATION IN THE SNIMED CODE 
    #So this table only gives cases from SRCode that are measured under Condition

In [ ]:
CREATE TABLE `CB_2107.Influenza_Cases_snomed_Cases_Record` AS
SELECT snomedcode, COUNT(*) AS RecordCount
FROM `CB_2107.Influenza_Cases_snomed_Cases`
GROUP BY snomedcode
ORDER BY snomedcode;
# This has SNOMED to person - ONLY Cases under Condiditon

In [ ]:
CREATE TABLE `CB_2107.Influenza_Cases_snomed_Cases_Record_joined` AS
SELECT A.*,B.*
FROM `CB_2107.Influenza_Cases_snomed_Cases_Record` A
INNER JOIN `CB_CDM_VOCAB.concept`B
ON A.snomedcode = B.concept_code;
# SNOMED TO person Mapping only condidtion = But includes SNOMED and Nebraska?

In [ ]:

CREATE TABLE `CB_2107.Influenza_Cases_snomed_Cases_Record_joined_SNOMED_ONLY` AS
SELECT *
FROM `CB_2107.Influenza_Cases_snomed_Cases_Record_joined`
WHERE (vocabulary_id = 'SNOMED' )
# This removed the dupliactions of SNOMED and Nabraska = Only SNOMED Kept 

In [ ]:
     SELECT *
FROM `CB_CDM_VOCAB.concept`  
WHERE (domain_id = 'Condition' or domain_id = 'Observation')

#Checking both Observation and Condition

In [ ]:
SELECT DISTINCT (ctv3text)
FROM `CB_FDM_PrimaryCare_V8.tbl_srcode`
WHERE (ctv3text LIKE '%COVID%' OR ctv3text LIKE '%Coronavirus%'OR ctv3text LIKE '%coronavirus%')
# Inclusion Criteria Tested 